In [ ]:
import sys
sys.path.insert(0, '../')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
import io
import tabulate
import helpers_eda_inicial
import helpers_cbc

Me traigo las datasets que estuve armando y limpiando. Ahora me gustaría poder estudiar los datos del CBC de las personas. Así luego resultará más fácil hacer el merge.

In [ ]:
personas_con_target = pd.read_csv('../../../assets/silver/data_utilizada/personas_con_target.csv')
actas_con_personas = pd.read_csv('../../../assets/silver/data_utilizada/personas_un_acta_por_fila.csv')
clusters = pd.read_csv('../../../assets/silver/CBC/df_carreras_con_cluster.csv')

In [ ]:
actas_con_personas['fecha'] = pd.to_datetime(actas_con_personas['fecha'], format='%Y-%m-%d')
actas_con_personas['dni'] = actas_con_personas['dni'].astype(str)
actas_con_personas.drop(columns = ['Unnamed: 0'], inplace = True)

In [ ]:
personas_con_target_dnis = personas_con_target['dni'].astype(str).unique()
personas_con_target_dnis.size

In [ ]:
personas_con_target.dtypes

Me traigo el cbc

In [ ]:
cbc, df_carreras = helpers_cbc.get_data('../../../assets/bronze/CBC/Sitacad_Tesis (1).xlsx')

In [ ]:
cbc.head(5)

Como ahora se el DNI de las personas con las uqe vamos a trabajar, me puedo quedar solo con esas filas.

In [ ]:
cbc.dtypes

Lo paso a string porque así está en personas_con_target pero vamos a revisar con cuidado los casos que no tienen match.

In [ ]:
cbc.shape

In [ ]:
cbc['Dni'] = cbc['Dni'].astype(str)

In [ ]:
cbc_en_fcen = cbc[cbc['Dni'].isin(personas_con_target_dnis)]

In [ ]:
cbc_en_fcen.shape, cbc_en_fcen['Dni'].nunique(), cbc_en_fcen['Dni'].nunique() / personas_con_target_dnis.size, cbc_en_fcen['Dni'].nunique() / cbc['Dni'].nunique()

In [ ]:
personas_con_target[~personas_con_target['dni'].isin(cbc_en_fcen['Dni'].unique())]['dni'].to_csv('../../../assets/silver/CBC/dnis_no_en_fcen.csv', index = False, header = False)

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(data=cbc_en_fcen, x='Fecha', bins=30)  # ajustá el número de bins si querés más o menos detalle
plt.title('Distribución de registros por fecha')
plt.xlabel('Fecha')
plt.ylabel('Cantidad de registros')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Veo que hay actas posteriores a 2022, por lo que me gustaría revisar qué características tienen las actas del CBC posteriores al año de inscripción de las personas (lo hago más abajo) Antes deamos la distribución de los meses de registro

In [ ]:
cbc_en_fcen['año'] = cbc_en_fcen['Fecha'].dt.year
cbc_en_fcen['mes'] = cbc_en_fcen['Fecha'].dt.month

In [ ]:
# Contar la cantidad de actas por año y mes
conteo = cbc_en_fcen.groupby(['año', 'mes']).size().reset_index(name='cantidad')

# Calcular el total de actas por año
totales_por_año = conteo.groupby('año')['cantidad'].transform('sum')

# Calcular el porcentaje
conteo['porcentaje'] = conteo['cantidad'] / totales_por_año * 100

# Asegurar el orden correcto de los meses si están en texto
#orden_meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
#               'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
#conteo['mes'] = pd.Categorical(conteo['mes'], categories=orden_meses, ordered=True)

# Crear el barplot agrupado
plt.figure(figsize=(12, 6))
sns.barplot(data=conteo, x='mes', y='cantidad', hue='año', palette='viridis')

plt.xlabel('Mes')
plt.ylabel('Cantidad de actas del CBC')
plt.title('Cantidad de actas por mes y año')
plt.xticks(rotation=45)
plt.legend(title='Año')
plt.tight_layout()
plt.show()

# Crear el barplot agrupado
plt.figure(figsize=(12, 6))
sns.barplot(data=conteo, x='mes', y='porcentaje', hue='año', palette='viridis')

plt.xlabel('Mes')
plt.ylabel('Porcentaje de actas del CBC(%)')
plt.title('Distribución porcentual de actas por mes y año')
plt.xticks(rotation=45)
plt.legend(title='Año')
plt.tight_layout()
plt.show()

In [ ]:
cbc_en_fcen[cbc_en_fcen['año']>2022]['año'].value_counts()

In [ ]:
cbc_en_fcen[cbc_en_fcen['año']>2022].groupby(['año', 'mes'])['Es materia FCEN?'].value_counts()

In [ ]:
cbc_en_fcen[cbc_en_fcen['año'] == 2025]

Sin personas que tienen registros de otros años, no hay problema con borrar los de 2025. Pero lo debo hacer en función de su cohorte.

In [ ]:
personas_con_target[personas_con_target['dni'].isin(cbc_en_fcen[cbc_en_fcen['año'] == 2025]['Dni'].unique())]

Debemos para cada dato del CBC asociar la cohorte de la persona. Anoto para cada cohorte, el mes y año del último registro que se puede tener del CBC.

* 2021C1: 09/2023
* 2021C2: 03/2024
* 2022C1: 09/2024
* 2022C2: 03/2025

In [ ]:
cbc_en_fcen.columns

In [ ]:
personas_con_target.columns

In [ ]:
cohorte_personas = personas_con_target[personas_con_target['dni'].isin(cbc_en_fcen['Dni'].unique())][['dni', 'cohorte']].drop_duplicates()
cohorte_personas.shape

In [ ]:
cbc_en_fcen = cbc_en_fcen.merge(cohorte_personas, how = 'left', left_on = 'Dni', right_on = 'dni')
cbc_en_fcen.drop(columns = ['dni'], inplace = True)
cbc_en_fcen.head(5)

In [ ]:
# Diccionario con los últimos registros válidos por cohorte
fechas_limite = {
    '2021C1': (2023, 9),
    '2021C2': (2024, 3),
    '2022C1': (2024, 9),
    '2022C2': (2025, 3)
}
def es_valido(row):
    año_limite, mes_limite = fechas_limite.get(row['cohorte'], (0, 0))
    if row['año'] < año_limite:
        return True
    elif row['año'] == año_limite:
        return row['mes'] <= mes_limite
    else:
        return False

cbc_en_fcen['valido'] = cbc_en_fcen.apply(es_valido, axis=1)

In [ ]:
cbc_en_fcen['valido'].value_counts()

In [ ]:
cbc_en_fcen[~cbc_en_fcen['valido']]['Es materia FCEN?'].value_counts()

In [ ]:
cbc_en_fcen = cbc_en_fcen[cbc_en_fcen['valido']]
cbc_en_fcen = cbc_en_fcen.drop(columns='valido')
cbc_en_fcen.head(5)

In [ ]:
cbc_en_fcen['Dni'].nunique()

In [ ]:
cbc_en_fcen.columns

Ahora que tengo separados a los que me "interesan". Puedo estudiar la distribución de algunas de las variables. En particular:

* 'Es materia FCEN?': Puedo ver la cantidad de personas que cursaron alguna materia que no sea de la FCEN
* 'Materia': Puedo ver la distribución de materias distintas que cursaron las personas
* 'Nota': Puedo ver la distribución del promedio de nota

Además, yo ya había calculado el tiempo de viaje con los datos del CBC, por lo que podría estudiarlo (aunque debería calcular tiempo de viaje de la FCEN también)

In [ ]:
cbc_en_fcen['Es materia FCEN?'].value_counts()

In [ ]:
sns.countplot(data=cbc_en_fcen, x='Es materia FCEN?')
plt.xlabel('Es materia FCEN?')
plt.ylabel('Cantidad de filas')
plt.title('Cantidad de filas por valor en "Es materia FCEN?"')
plt.show()

In [ ]:
# Filtrar los dni que tienen al menos una fila con 'SI'
dni_con_si = cbc_en_fcen[cbc_en_fcen['Es materia FCEN?'] == 'SI']['Dni'].unique()

# Crear un nuevo DataFrame para graficar
resumen = pd.DataFrame({
    'Tiene al menos un SI': ['Sí', 'No'],
    'Cantidad de dni': [
        len(dni_con_si),
        cbc_en_fcen['Dni'].nunique() - len(dni_con_si)
    ]
})

# Hacer el gráfico
sns.barplot(data=resumen, x='Tiene al menos un SI', y='Cantidad de dni')
plt.title('Cantidad de DNI según si tienen al menos un "SI" en "Es materia FCEN?"')
plt.show()


In [ ]:
resumen

In [ ]:
cbc_en_fcen[~cbc_en_fcen['Dni'].isin(dni_con_si)]

In [ ]:
personas_con_target[personas_con_target['dni'].isin(cbc_en_fcen[~cbc_en_fcen['Dni'].isin(dni_con_si)]['Dni'].unique())]['año_inscripcion_facultad'].value_counts()

In [ ]:
# Calcular proporciones
proporciones_filas = cbc_en_fcen['Materia'].value_counts().reset_index()
proporciones_filas.columns = ['Materia', 'Cantidad']

sns.barplot(data=proporciones_filas, x='Materia', y = 'Cantidad')
plt.xlabel('Materia')
plt.ylabel('Cantidad de filas')
plt.title('Cantidad de filas por Materia')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Calcular proporciones
proporciones_filas = cbc_en_fcen['Materia'].value_counts(normalize=True).reset_index()
proporciones_filas.columns = ['Materia', 'Proporción']

# Gráfico
sns.barplot(data=proporciones_filas, x='Materia', y='Proporción')
plt.title('Proporción de filas por valor en "Materia"')
plt.ylabel('Proporción')
plt.xticks(rotation=90)
plt.show()

Me gustaría ver la proporción de los clusters, ya cuento con un archivo donde asigne un número a cada uno.

In [ ]:
clusters = clusters[['Materia', 'Cluster']].drop_duplicates()
clusters.columns = ['materia', 'cluster']

In [ ]:
clusters.sort_values(by = 'cluster')

In [ ]:
cbc_en_fcen = cbc_en_fcen.merge(clusters, how = 'left', left_on = 'Materia', right_on = 'materia')

Los que no tienen valores asignados es porque son materias de otras carreras

In [ ]:
cbc_en_fcen.isna().sum()

In [ ]:
cbc_en_fcen.loc[cbc_en_fcen['Materia'] == 'ALGEBRA A', 'cluster'] = 5

In [ ]:
cbc_en_fcen[cbc_en_fcen['materia'].isna()]['Es materia FCEN?'].unique()

In [ ]:
cbc_en_fcen.drop(columns = ['materia'], inplace = True)
cbc_en_fcen['cluster'].fillna(8, inplace = True)

In [ ]:
cbc_en_fcen

In [ ]:
# Calcular proporciones
proporciones_filas = cbc_en_fcen['cluster'].value_counts().reset_index()
proporciones_filas.columns = ['cluster', 'Cantidad']

sns.barplot(data=proporciones_filas, x='cluster', y = 'Cantidad')
plt.xlabel('cluster')
plt.ylabel('Cantidad de filas')
plt.title('Cantidad de filas por cluster')
plt.xticks(rotation=0)
plt.show()

In [ ]:
# Calcular proporciones
proporciones_filas = cbc_en_fcen['cluster'].value_counts(normalize=True).reset_index()
proporciones_filas.columns = ['cluster', 'Proporción']

# Gráfico
sns.barplot(data=proporciones_filas, x='cluster', y='Proporción')
plt.title('Proporción de filas por valor en "cluster"')
plt.ylabel('Proporción')
plt.xticks(rotation=0)
plt.show()

In [ ]:
# Calcular proporciones
proporciones_filas = cbc_en_fcen['Nota'].value_counts().reset_index()
proporciones_filas.columns = ['Nota', 'Cantidad']

sns.barplot(data=proporciones_filas, x='Nota', y = 'Cantidad')
plt.xlabel('Nota')
plt.ylabel('Cantidad de filas')
plt.title('Cantidad de filas por Nota')
plt.xticks(rotation=0)
plt.show()

In [ ]:
# Calcular proporciones
proporciones_filas = cbc_en_fcen['Nota'].value_counts(normalize=True).reset_index()
proporciones_filas.columns = ['Nota', 'Proporción']

# Gráfico
sns.barplot(data=proporciones_filas, x='Nota', y='Proporción')
plt.title('Proporción de filas por valor en "Nota"')
plt.ylabel('Proporción')
plt.xticks(rotation=0)
plt.show()

## Veo los datos de la FCEN (mas que nada para ver la condición de los estudiantes de los que sí tenemos registro del CBC)

In [ ]:
personas_con_target_y_cbc = personas_con_target[personas_con_target['dni'].isin(cbc_en_fcen['Dni'].unique())]

In [ ]:
# Agrupar y contar
df_plot = (
    personas_con_target_y_cbc
    .groupby(['cohorte', 'target'])
    .size()
    .reset_index(name='count')
)

# Crear barplot
plt.figure(figsize=(10, 6))
sns.barplot(data=df_plot, x='cohorte', y='count', hue='target')
plt.title('Distribución de Target por Cohorte')
plt.xlabel('Cohorte')
plt.ylabel('Cantidad')
plt.legend(title='Target')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Paso 1: Conteo de cada (cohorte, target)
conteos = (
    personas_con_target_y_cbc
    .groupby(['cohorte', 'target'])
    .size()
    .reset_index(name='count')
)

# Paso 2: Calcular proporciones por cohorte
conteos['proporcion'] = (
    conteos
    .groupby('cohorte')['count']
    .transform(lambda x: x / x.sum())
)

# Paso 3: Plotear
plt.figure(figsize=(10, 6))
sns.barplot(data=conteos, x='cohorte', y='proporcion', hue='target')
plt.title('Proporción de Target por Cohorte')
plt.xlabel('Cohorte')
plt.ylabel('Proporción')
plt.legend(title='Target')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Voy a revisar qué características tienen las actas del CBC posteriores al año de inscripción de las personas.

In [ ]:
personas_con_target_y_cbc.columns

## Con la target

podemos ver cómo es la distribución de algunas de las variables condicionando por la target.

In [ ]:
cbc_en_fcen.head(5)

In [ ]:
cbc_en_fcen['Fecha'].describe()

In [ ]:
cbc_en_fcen[cbc_en_fcen['UBA XXI'] == 'SI']['Materia'].unique()

In [ ]:
cbc_en_fcen[cbc_en_fcen['UBA XXI'] == 'SI']['cluster'].unique()

In [ ]:
cbc_en_fcen.columns

In [ ]:
print(cbc_en_fcen.groupby(['cohorte', 'Es materia FCEN?'])['Fecha'].value_counts().reset_index(name='count').sort_values(by=['cohorte', 'Es materia FCEN?', 'Fecha']).to_markdown())

como esto despues lo voy a usar para entrenar el modelo, no puedo tener los registros del semestre relativo 4 --> los saco (vemos que no son muchos igual)

In [ ]:
# Diccionario con los últimos registros válidos por cohorte
fechas_limite = {
    '2021C1': (2023, 2),
    '2021C2': (2023, 8),
    '2022C1': (2024, 2),
    '2022C2': (2024, 8)
}
def es_valido(row):
    año_limite, mes_limite = fechas_limite.get(row['cohorte'], (0, 0))
    if row['año'] < año_limite:
        return True
    elif row['año'] == año_limite:
        return row['mes'] <= mes_limite
    else:
        return False

cbc_en_fcen['valido'] = cbc_en_fcen.apply(es_valido, axis=1)

In [ ]:
cbc_en_fcen = cbc_en_fcen[cbc_en_fcen['valido']]
cbc_en_fcen['Dni'].nunique()

In [ ]:
cbc_en_fcen[cbc_en_fcen['Nota'] == 'AP'].shape

In [ ]:
cbc_en_fcen[cbc_en_fcen['Dni'].isin(cbc_en_fcen[cbc_en_fcen['Nota'] == 'AP']['Dni'].unique())].groupby(['Dni']).size()

In [ ]:
# Suponemos que df ya tiene las columnas: DNI, Cluster, Nota
cbc_en_fcen.loc[:,'Nota'] = cbc_en_fcen['Nota'].replace('A', '0')
cbc_en_fcen.loc[:,'Nota'] = cbc_en_fcen['Nota'].replace('NR', '0')

In [ ]:
#elimino filas que tienen AP en la columna 'Nota'
# reemplazo los AP en la columna Nota por el promedio de la persona
cbc_en_fcen['nota_num'] = pd.to_numeric(cbc_en_fcen['Nota'], errors='coerce')  # Convertir a numérico, NaN para errores

In [ ]:
cbc_en_fcen['nota_num'].isnull().sum()

In [ ]:
# Calculamos el promedio por persona ignorando 'AP' (NaN)
promedios = np.floor(cbc_en_fcen.groupby('Dni')['nota_num'].mean())

# Función para reemplazar 'AP' con el promedio correspondiente
def reemplazar_ap(row):
    if row['Nota'] == 'AP':
        return promedios[row['Dni']]
    else:
        return row['nota_num']

# Aplicamos la función
cbc_en_fcen['nota_limpia'] = cbc_en_fcen.apply(reemplazar_ap, axis=1)
# Si querés, podés reemplazar la columna original o eliminar las auxiliares
cbc_en_fcen['Nota'] = cbc_en_fcen['nota_limpia']
cbc_en_fcen = cbc_en_fcen.drop(columns=['nota_num', 'nota_limpia'])

#paso la columna a integer
cbc_en_fcen.loc[:,'Nota'] = cbc_en_fcen['Nota'].astype(int)

In [ ]:
cbc_en_fcen['Nota'].unique()

In [ ]:
# Suponiendo que ya tenés el DataFrame cargado como df
fecha_cero = pd.Timestamp("2020-01-01")
cbc_en_fcen['Fecha_numerica'] = (cbc_en_fcen['Fecha'] - fecha_cero).dt.days
cbc_en_fcen['Fecha_numerica'].min(), cbc_en_fcen['Fecha_numerica'].max()

In [ ]:
# Agrupamos
agg = cbc_en_fcen.groupby(['Dni', 'cluster']).agg(
    nota_promedio=('Nota', 'mean'),
    fecha_promedio=('Fecha_numerica', 'mean')
).reset_index()

In [ ]:
# Paso 1: Agrupar para saber cuántas veces rindió cada materia
rindio_por_materia = cbc_en_fcen.groupby(['Dni', 'Materia', 'cluster']).agg(
    veces_rindio=('Nota', 'count')
).reset_index()

# Paso 2: Agrupar por Dni y cluster, y calcular el promedio de "veces que rindió por materia"
promedio_rindio_por_cluster = rindio_por_materia.groupby(['Dni', 'cluster']).agg(
    cantidad_veces_promedio=('veces_rindio', 'mean')
).reset_index()
# Paso 2: juntamos con promedio de veces que rindió por materia dentro del cluster
agg_final = agg.merge(promedio_rindio_por_cluster, on=['Dni', 'cluster'], how='left')

In [ ]:
agg_final[agg_final['cluster'] == 7]

In [ ]:
# Pivot nota
pivot_nota = agg_final.pivot(index='Dni', columns='cluster', values='nota_promedio')
pivot_nota.columns = [f'nota_cluster_{col}' for col in pivot_nota.columns]

# Pivot cantidad de veces que rindió
pivot_cantidad = agg_final.pivot(index='Dni', columns='cluster', values='cantidad_veces_promedio')
pivot_cantidad.columns = [f'cantidad_de_veces_que_rindio_cluster_{col}' for col in pivot_cantidad.columns]

# Pivot fecha
pivot_fecha = agg_final.pivot(index='Dni', columns='cluster', values='fecha_promedio')
pivot_fecha.columns = [f'fecha_promedio_cluster_{col}' for col in pivot_fecha.columns]

# Unimos todo
df_final = pd.concat([pivot_nota, pivot_cantidad, pivot_fecha], axis=1).reset_index()

In [ ]:
df_final

In [ ]:
helpers_eda_inicial.display_missing_values(df_final)

voy a probar otra forma de distribuir los clusters, a ver si mejora:

In [ ]:
cbc_en_fcen.columns

In [ ]:
cbc_en_fcen_copia = cbc_en_fcen[['Dni', 'Nota', 'Materia', 'Fecha_numerica', 'cluster', 'UBA XXI']].copy()
cbc_en_fcen_copia.loc[cbc_en_fcen_copia['Materia'] == 'ALGEBRA A', 'cluster'] = 1
cbc_en_fcen_copia.loc[cbc_en_fcen_copia['Materia'] == 'ALGEBRA', 'cluster'] = 1
cbc_en_fcen_copia.loc[cbc_en_fcen_copia['Materia'] == 'INTROD. AL PENSAMIENTO COMPUTACIONAL', 'cluster'] = 4

In [ ]:
# Agrupamos
agg = cbc_en_fcen_copia.groupby(['Dni', 'cluster']).agg(
    nota_promedio=('Nota', 'mean'),
    cantidad_rindio=('Nota', 'count'),
    fecha_promedio=('Fecha_numerica', 'mean')
).reset_index()

# Paso 1: Agrupar para saber cuántas veces rindió cada materia
rindio_por_materia = cbc_en_fcen_copia.groupby(['Dni', 'Materia', 'cluster']).agg(
    veces_rindio=('Nota', 'count')
).reset_index()

# Paso 2: Agrupar por Dni y cluster, y calcular el promedio de "veces que rindió por materia"
promedio_rindio_por_cluster = rindio_por_materia.groupby(['Dni', 'cluster']).agg(
    cantidad_veces_promedio=('veces_rindio', 'mean')
).reset_index()
# Paso 2: juntamos con promedio de veces que rindió por materia dentro del cluster
agg_final = agg.merge(promedio_rindio_por_cluster, on=['Dni', 'cluster'], how='left')

# Pivot nota
pivot_nota = agg_final.pivot(index='Dni', columns='cluster', values='nota_promedio')
pivot_nota.columns = [f'nota_cluster_{col}' for col in pivot_nota.columns]

# Pivot cantidad de veces que rindió
pivot_cantidad = agg_final.pivot(index='Dni', columns='cluster', values='cantidad_veces_promedio')
pivot_cantidad.columns = [f'cantidad_de_veces_que_rindio_cluster_{col}' for col in pivot_cantidad.columns]

# Pivot fecha
pivot_fecha = agg_final.pivot(index='Dni', columns='cluster', values='fecha_promedio')
pivot_fecha.columns = [f'fecha_promedio_cluster_{col}' for col in pivot_fecha.columns]

# Unimos todo
df_final = pd.concat([pivot_nota, pivot_cantidad, pivot_fecha], axis=1).reset_index()


In [ ]:
uba_xxi_flags = cbc_en_fcen_copia.groupby(['Dni', 'cluster'])['UBA XXI'].apply(
    lambda x: any(v == 'SI' for v in x)
).reset_index().rename(columns={'UBA XXI': 'cursada_por_uba_xxi'})

pivot_uba_xxi = uba_xxi_flags.pivot(index='Dni', columns='cluster', values='cursada_por_uba_xxi')
pivot_uba_xxi.columns = [f'cursada_por_uba_xxi_cluster_{col}' for col in pivot_uba_xxi.columns]

# Primero reseteamos el índice para tener 'Dni' como columna
pivot_uba_xxi = pivot_uba_xxi.reset_index()

# Luego mergeamos
df_final = df_final.merge(pivot_uba_xxi, on='Dni', how='left')

In [ ]:
df_final.columns

In [ ]:
df_final.shape

In [ ]:
helpers_eda_inicial.display_missing_values(df_final)

In [ ]:
# Calcular proporciones
proporciones_filas = cbc_en_fcen_copia['cluster'].value_counts().reset_index()
proporciones_filas.columns = ['cluster', 'Cantidad']

sns.barplot(data=proporciones_filas, x='cluster', y = 'Cantidad')
plt.xlabel('cluster')
plt.ylabel('Cantidad de filas')
plt.title('Cantidad de filas por cluster')
plt.xticks(rotation=0)
plt.show()

# Calcular proporciones
proporciones_filas = cbc_en_fcen_copia['cluster'].value_counts(normalize=True).reset_index()
proporciones_filas.columns = ['cluster', 'Proporción']

# Gráfico
sns.barplot(data=proporciones_filas, x='cluster', y='Proporción')
plt.title('Proporción de filas por valor en "cluster"')
plt.ylabel('Proporción')
plt.xticks(rotation=0)
plt.show()

In [ ]:
# Set estilo
sns.set(style="whitegrid")
variables_creadas = df_final.drop(columns='Dni')

# Gráficos
variables_creadas.hist(bins=20, figsize=(20, 15))
plt.suptitle("Distribución de todas las variables por cluster", fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Seleccionamos las columnas
cols_nota = [f'nota_cluster_{i}.0' for i in range(1, 7)] + [f'nota_cluster_{i}.0' for i in range(8, 9)]
df_notas = df_final[cols_nota]

# Filtramos columnas que no sean completamente NaN
df_notas = df_notas.loc[:, df_notas.notna().any()]

# Convertimos a numérico por si acaso (ignora errores)
df_notas = df_notas.apply(pd.to_numeric, errors='coerce')

# Graficamos
sns.set(style="whitegrid")
df_notas.hist(bins=10, figsize=(16, 10))
plt.suptitle("Distribución de notas promedio por cluster (1.0 al 8.0)", fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Filtrar columnas de interés
uba_xxi_columns = [col for col in df_final.columns if col.startswith('cursada_por_uba_xxi_cluster_')]

# Calcular conteos por columna
value_counts_df = pd.DataFrame({
    col: df_final[col].value_counts(dropna=False) for col in uba_xxi_columns
}).T.fillna(0)

# Asegurar que todos los valores (True, False, NaN) estén como columnas
for val in [True, False, np.nan]:
    if val not in value_counts_df.columns:
        value_counts_df[val] = 0

# Ordenar columnas
value_counts_df = value_counts_df[[True, False, np.nan]]

# Renombrar columnas para mejor lectura
value_counts_df.columns = ['Cursó por UBA XXI', 'No cursó por UBA XXI', 'Sin datos']

# Graficar
value_counts_df.plot(kind='bar', stacked=False, figsize=(12, 6), colormap='viridis')
plt.title('Distribución de cursada por UBA XXI por cluster')
plt.ylabel('Cantidad de estudiantes')
plt.xlabel('Cluster')
plt.xticks(rotation=45, ha='right')
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()


In [ ]:
# Filtrar columnas de interés
uba_xxi_columns = [col for col in df_final.columns if col.startswith('cursada_por_uba_xxi_cluster_')]

# Calcular conteos por columna
value_counts_df = pd.DataFrame({
    col: df_final[col].value_counts(dropna=False) for col in uba_xxi_columns
}).T.fillna(0)

# Asegurar que todos los valores estén como columnas
for val in [True, False, np.nan]:
    if val not in value_counts_df.columns:
        value_counts_df[val] = 0

# Ordenar columnas
value_counts_df = value_counts_df[[True, False, np.nan]]

# Renombrar columnas para legibilidad
value_counts_df.columns = ['Cursó por UBA XXI', 'No cursó por UBA XXI', 'Sin datos']

# Calcular proporciones
proportions_df = value_counts_df.div(value_counts_df.sum(axis=1), axis=0)

# Graficar
proportions_df.plot(kind='bar', stacked=False, figsize=(12, 6), colormap='viridis')
plt.title('Proporción de estudiantes que cursaron por UBA XXI por cluster')
plt.ylabel('Proporción')
plt.xlabel('Cluster')
plt.xticks(rotation=45, ha='right')
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
personas_con_target_y_cbc = personas_con_target_y_cbc.merge(df_final, how = 'left', left_on = 'dni', right_on = 'Dni')

In [ ]:
# Cantidad de filas por cluster y target
cantidades = personas_con_target_y_cbc.groupby(['nota_cluster_1.0', 'target']).size().reset_index(name='Cantidad')

# Gráfico
sns.barplot(data=cantidades, x='nota_cluster_1.0', y='Cantidad', hue='target')
plt.xlabel('nota_cluster_1.0')
plt.ylabel('Cantidad de filas')
plt.title('Cantidad de filas por cluster y target')
plt.xticks(rotation=90)
plt.show()

In [ ]:
df_final.dtypes

In [ ]:
for col in df_final.columns:
    if 'nota_cluster' in col or 'cantidad_de_veces_que_rindio_cluster' in col or 'fecha_promedio_cluster' in col:
        df_final[col] = np.floor(df_final[col])

## Nos guardamos la versión final!

In [ ]:
df_final.to_csv('../../../assets/silver/data_utilizada/cbc_final.csv', index = False)

In [ ]:
for col in cols_nota:
    personas_con_target_y_cbc[col] = pd.to_numeric(
        personas_con_target_y_cbc[col], errors='coerce'
    )

In [ ]:
personas_con_target_y_cbc['dni'].nunique()

Estan buenos estos graficos, pero en todo caso los debería hacer cuando haga merge con todo el resto, si no medio raro

In [ ]:
def plot_histogram(cluster):

    # Agrupar por bin y target
    cantidades = personas_con_target_y_cbc.groupby([f'nota_cluster_{cluster}.0', 'target']).size().reset_index(name='Cantidad')

    # Gráfico de barras con cantidades incluyendo NA
    plt.figure(figsize=(10, 6))
    sns.barplot(data=cantidades, x=f'nota_cluster_{cluster}.0', y='Cantidad', hue='target')

    plt.xlabel(f'nota_cluster_{cluster}.0')
    plt.ylabel('Cantidad de filas')
    plt.title(f'Cantidad de filas por nota_cluster_{cluster}.0 (incluyendo NA) y target')
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()


    # Calcular la suma total por bin para calcular proporciones
    total_por_bin = cantidades.groupby(f'nota_cluster_{cluster}.0')['Cantidad'].transform('sum')

    # Calcular la proporción por cada grupo target dentro del bin
    cantidades['Proporcion'] = cantidades['Cantidad'] / total_por_bin

    # Gráfico de barras con proporciones incluyendo NA
    plt.figure(figsize=(10, 6))
    sns.barplot(data=cantidades, x=f'nota_cluster_{cluster}.0', y='Proporcion', hue='target')

    plt.xlabel(f'nota_cluster_{cluster}.0')
    plt.ylabel('Proporción')
    plt.title(f'Proporción por nota_cluster_{cluster}.0 (incluyendo NA) y target')
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()


In [ ]:
for i in range(1, 7):
    plot_histogram(i)

In [ ]:
plot_histogram(8)

In [ ]:
def plot_histogram(cluster):
    columna = f'nota_cluster_{cluster}.0'

    # Crear una copia con NA reemplazado por string 'NA'
    df = personas_con_target_y_cbc.copy()
    df[columna] = df[columna].fillna('NA')

    # Agrupar por bin (incluyendo 'NA') y target
    cantidades = df.groupby([columna, 'target']).size().reset_index(name='Cantidad')

    # Gráfico de barras con cantidades incluyendo NA
    plt.figure(figsize=(10, 6))
    sns.barplot(data=cantidades, x=columna, y='Cantidad', hue='target')
    plt.xlabel(columna)
    plt.ylabel('Cantidad de filas')
    plt.title(f'Cantidad de filas por {columna} (incluyendo NA) y target')
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()

    # Calcular la suma total por bin para calcular proporciones
    total_por_bin = cantidades.groupby(columna)['Cantidad'].transform('sum')

    # Calcular la proporción por cada grupo target dentro del bin
    cantidades['Proporcion'] = cantidades['Cantidad'] / total_por_bin

    # Gráfico de barras con proporciones incluyendo NA
    plt.figure(figsize=(10, 6))
    sns.barplot(data=cantidades, x=columna, y='Proporcion', hue='target')
    plt.xlabel(columna)
    plt.ylabel('Proporción')
    plt.title(f'Proporción por {columna} (incluyendo NA) y target')
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()


In [ ]:
for i in range(1, 7):
    plot_histogram(i)

In [ ]:
plot_histogram(8)